In [7]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

import matplotlib.animation as animation

mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

n_epochs = 20   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 200    # Size of the train dataset
n_test = 120     # Size of the test dataset
n_batches = 4     # Size of the batches

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

n_qubits = 4

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnotnode(inputs):
    
    for j in range(4):
        qml.RY(np.pi * inputs[j], wires=j)


    # Filter from arxiv.org/abs/2308.14930

    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 3])


    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]

def quanv(image, func):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = func(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

anim = ""    
anims = []

ims = []
    
def vizquanv(inputs, func, tittle):

    out = np.zeros((14, 14, 4))
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(12, 4))
    for c in range(n_qubits):
        ims.append([[]])
        ax[c].set_title("{tittle} [ch. {c}]".format(tittle=tittle,c=c), fontsize=16)
        ax[c].axis('off')
        ax[c].imshow(quanv(inputs, func)[:,:,c])

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = func(
                [
                    inputs[j, k, 0],
                    inputs[j, k + 1, 0],
                    inputs[j + 1, k, 0],
                    inputs[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(n_qubits):
                out[j // 2, k // 2, c] = q_results[c]
                im = ax[c].imshow(out[:,:,c], animated=True)
                ims[c].append([im])
    for c in range(n_qubits):
        ims[c].insert(0,ims[c][-1])
        anim = animation.ArtistAnimation(fig, ims[c], interval=100, blit=True,
                                repeat_delay=9000)
        anims.append([anim])
        
    return out

In [8]:
 
%matplotlib widget

ou = vizquanv(train_images[0], qnotnode, "Five")

#plt.tight_layout()
plt.show()


In [9]:
ou.shape

(14, 14, 4)